# Exploration

Dans cette partie on va donc explorer nos données afin de se familiariser avec le dataset.

Pour cette exploration j'avais au départ utilisé un attribut `version` qui me permettait d'avoir le même résultat par rapport à la version du script que je souhaite utiliser. Mais j'ai changé d'avis car je commençais à avoir trop de blocks "if" pour checker la version du code à utiliser.

En interne certaines actions étaient donc effectuées sur le dataset suivant la version. Cela me permettait de montrer ma démarche et la progression des traitements / explorations du dataset. Néanmoins maintenant je vais donc montrer les changements au fur et à mesure du notebook.

Je montre donc à travers les champs `markdown` les moments où j'ai testé des fonctions. J'ai ensuite implémenté ces fonctions dans mon script python `src/dataset/dataset.py`

In [ ]:
%load_ext autoreload
%autoreload 2
from src.dataset import Dataset
import matplotlib.pyplot as plt
import pandas as pd
import cudf
import numpy as np

plt.style.use(['ggplot', 'https://raw.githubusercontent.com/AlanBlanchet/matplotlib_styles/master/vscode_blue.mplstyle'])

Dataset.init()

On peut faire une cellule de comparaison pour avoir un affichage des données brutes et des données traitées

In [ ]:
topics = Dataset("topics1.csv")
topics.df.head()

In [ ]:
topics.df["text"].head()

In [ ]:
topics.example(interactive=False, index=0)

J'avais donc au départ effectué plusieurs opération avec un système de version. Voici ce qui avait été fait :

- Remarquer les poncutation et les traiter
- Mise en place d'un système pour voir des examples (comme ci-dessus) mais de manière intéractive
- Remarquer la négation "n't" qui est présente
- Des mots qu'on aimerait peut être exclure car il n'apporte pas de plus value pour nos prédictions de tags comme par exemple : "problem", "error", "get", "code", "like"...
    - Pour régler le problème de la négation, je me suis d'abord dit qu'il serait bien de supprimer les chaînes qui ont une longueur égale à 1. Mais rappelons un instant que le langage C comporte 1 seul caractère. Il n'est donc pas possible de procéder comme cela. On aimerait possiblement conserver la négation car cela pourrait être utile pour une phrase du genre : "this is not Java". On gardera dans un premier temps cette négation.
    - Pour les mots qu'on aimerait exclure, je propose de mettre ces mots dans un fichier texte. Ils feront office de "stopwords" propres au projet (dans data/exclude.txt). On fera ce traitement si besoin par la suite 

Regardons les mots les plus communs et identifions les mots inutiles

In [ ]:
topics.example(index=1)

On a donc tous les outils en main pour commencer à effectuer des analyses plus poussées et effectuer des tests sur nos données.

In [ ]:
sententences_l = topics.df["text"].str.len().sort_values()
plt.figure(figsize=(8,5))
plt.title("Text indece by their lengths")
plt.xlabel("Text indice")
plt.ylabel("Text length")
sententences_l.reset_index(drop=True).plot()
sententences_l.describe(percentiles=[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975])

On remarque que quelques phrases sont très longues (2.5% avec l > 6992)

In [ ]:
target_list = topics.df[topics.targets].stack().reset_index().drop(columns="level_1").set_index("level_0")
target_list = target_list[0].astype(str)

In [ ]:
target_list_counts = target_list.value_counts().sort_values()
target_list_counts_tail = target_list_counts.tail(30)
target_list_counts_tail.plot.barh()
plt.title("Top 30 most commonly used tags in the dataset")
plt.xlabel("Counts")
plt.ylabel("Targets")

In [ ]:
target_list_count_under_5 = target_list_counts[target_list_counts < 5]
target_list_count_under_5

Certains targets ne seront certainement pas simple à trouver car seulement 5000 ont une fréquence de 5 ou +. On peut donc se demander si conserver autant d'éléments est utile. On verra par la suite ce que l'on fait

In [ ]:
target_list_over_5 = target_list[~target_list.isin(target_list_count_under_5.index)]
target_list_over_5_index_counts = cudf.Series(target_list_over_5.index).value_counts()
target_list_over_5_target_count = target_list_over_5_index_counts.value_counts()
target_list_over_5_target_count[0] = len(topics.df) - target_list_over_5_target_count.sum()
plt.figure(figsize=(8,5))
plt.title("Leftover targets number per text\nafter removing low frequency targets (< 5)")
plt.yscale('log')
target_list_over_5_target_count.to_pandas().plot(kind="bar")
plt.xlabel("Number of leftovers")
plt.ylabel("Number of texts")
target_list_over_5_target_count

De plus, si on retirait ces targets, 7 lignes n'auraient plus de target du tout.

# Preprocessing avancé

## Itération 1

Dans cette partie on va regarder un peu plus notre dataset pour trouver des éléments mal preprocessé et noter l'index des phrases où l'on trouve ces cas

In [ ]:
topics.example(index=1)

J'ai effectué ce traitement sur mon dataset local qui, en plus, a changé entre temps. Ce qui compte malgré tout sont les problèmes que j'ai pu y repérer

Voici donc les élément que nous devons maintenant traiter ainsi que l'index des phrases où le problème a été repéré :

- Les nombres (2, 14)
- Le "#" de C# est coupé (6)
- Attention à ne pas supprimer délaisser des nombres importants comme 64 (8)
- Dates peuvent être importantes (8)
- Le "++" de C++ est coupé (10,30)
- Il y a des urls (12,19,26,34,35,52,54)
- Des morceaux de code sans block de code (12,13,26,27,52,500)
- On supprime les balises "code" ce qui a pour effet de laisser certains textes un peu trop vide (14,19,20)

Constats :
- Certains éléments sont très compliqués à tagger même pour un être humain (9,11,500)
- La négation est importants dans certains contextes (10)
- Certains tags seront quasiement introuvable pour certaines situations. Il faudra habilement choisir une fonction de score (tous les exemples)

On remarque que beaucoup de targets ont des version (ex: "c++-32") associées à leur nom. J'ai bien peur que le fait que la version soit présente brouille la prédiction.

On se rappellera d'un possible traitement à effectuer par la suite. Je suppose naivement que l'on ne souhaite pas forcément prédire la version spécifique en fonction d'une question mais plutôt juste le tag initial

# Itération 2

La prochaine étape serait de s'intéresser à la balise code afin de rajouter de l'information.
En effet, comme on l'a vu précédemment, certaines questions on vu la majorité de contenu se faire supprimer car la majorité était du code.

On peut commencer par essayer, en fonction de la balise de code, de détecter le language qui y est présent.

Après expérimentation avec les packages [guesslang](https://github.com/yoeo/guesslang) / [whats_that_code](https://github.com/matthewdeanmartin/whats_that_code) c'était très difficile. Le seule package qui m'apporte un peu d'espoir est [pygments](https://github.com/pygments/pygments) mais il a parfois de mauvaises prédictions.

Je propose malgré tout de l'utiliser et de voir ce que cela donne.

On va donc simplement remplacer la balise de code par le nom du langage détecté pour chaque morceaux de code.

D'après ce que je vois, ce n'est pas fameux.

Supprimons les lettres uniques et la négation pour le moment sans enlever la lettre C qui correspond au langage.

Je me dis qu'on peut certainement conserver la balise code ainsi que son contenu et espère que notre IA se rende compte que certains mots clés appartiennent à un langage spécifique (ex: "def" en python).

Il y aura certainement encore des traitements à faire car dans le code il peut se passer plein de choses => Commentaires / Caractères spéciaux etc...

### Conclusion :

J'avais donc déjà testé cela avec mon système de version.

J'ai décidé de ne pas continuer avec cette approche car elle était trop instable. Je n'étais également pas très à l'aise avec le fait de rajouter manuellement ce que le modèle pensait être le langage à la fin de texte. Les prédictions n'étaient également pas forcément tout le temps bonnes et il n'est pas forcément tout le temps question de langage de programmation mais par exemple de technologies / librairies.

# Itération 3

On va dans cette partie s'intéresser aux targets. En effet cette dernière possède parfois des détails peu pertinents comme par exemple des versions trop spécifiques.

Il faut également formatter la target pour qu'elle fonctionne bien avec le reste du code. Certains modèles prennent en entrée des caractères limités. Par exemple sans le caractère '+' présent dans 'c++'. Il faut donc encoder ce string en 'cpp' par exemple.

En faisant mes recherches j'ai trouvé ce string correspondant aux target à l'intérieur de mon csv :

`<android><android-mediaplayer><audio-player><monkey><android-monkey><android-music-player>`

On peut voir qu'il y a 6 targets au lieux de 5. J'ai donc traiter ce cas en limitant à 5 targets tout simplement.

Regardons quelques données pour voir si on peut traiter certains cas absurdes

On va également les trier par ordre de fréquence. Target 1 aura la target la plus fréquente et target 5 la moins fréquente de notre distribution de targets